In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/wordnet/wordnet/data.adj
/kaggle/input/wordnet/wordnet/LICENSE
/kaggle/input/wordnet/wordnet/index.adj
/kaggle/input/wordnet/wordnet/README
/kaggle/input/wordnet/wordnet/verb.exc
/kaggle/input/wordnet/wordnet/index.adv
/kaggle/input/wordnet/wordnet/index.verb
/kaggle/input/wordnet/wordnet/data.verb
/kaggle/input/wordnet/wordnet/lexnames
/kaggle/input/wordnet/wordnet/data.noun
/kaggle/input/wordnet/wordnet/noun.exc
/kaggle/input/wordnet/wordnet/index.noun
/kaggle/input/wordnet/wordnet/adj.exc
/kaggle/input/wordnet/wordnet/index.sense
/kaggle/input/wordnet/wordnet/adv.exc
/kaggle/input/wordnet/wordnet/cntlist.rev
/kaggle/input/wordnet/wordnet/data.adv
/kaggle/input/wordnet/wordnet/citation.bib
/kaggle/input/wordnet/wordnet/wordnet/data.adj
/kaggle/input/wordnet/wordnet/wordnet/LICENSE
/kaggle/input/wordnet/wordnet/wordnet/index.adj
/kaggle/input/wordnet/wordnet/wordnet/README
/kaggle/input/wordnet/wordnet/wordnet/verb.exc
/kaggle/input/wordnet/wordnet/wordnet/index.adv
/kag

In [218]:
import torch
from torch.nn import functional as F
import pandas as pd
import nltk
from nltk import WordNetLemmatizer, WordPunctTokenizer
import matplotlib.pyplot as plt

In [3]:
nltk.download('wordnet')
!unzip /usr/share/nltk_data/corpora/wordnet.zip -d /usr/share/nltk_data/corpora/

[nltk_data] Downloading package wordnet to /usr/share/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
Archive:  /usr/share/nltk_data/corpora/wordnet.zip
   creating: /usr/share/nltk_data/corpora/wordnet/
  inflating: /usr/share/nltk_data/corpora/wordnet/lexnames  
  inflating: /usr/share/nltk_data/corpora/wordnet/data.verb  
  inflating: /usr/share/nltk_data/corpora/wordnet/index.adv  
  inflating: /usr/share/nltk_data/corpora/wordnet/adv.exc  
  inflating: /usr/share/nltk_data/corpora/wordnet/index.verb  
  inflating: /usr/share/nltk_data/corpora/wordnet/cntlist.rev  
  inflating: /usr/share/nltk_data/corpora/wordnet/data.adj  
  inflating: /usr/share/nltk_data/corpora/wordnet/index.adj  
  inflating: /usr/share/nltk_data/corpora/wordnet/LICENSE  
  inflating: /usr/share/nltk_data/corpora/wordnet/citation.bib  
  inflating: /usr/share/nltk_data/corpora/wordnet/noun.exc  
  inflating: /usr/share/nltk_data/corpora/wordnet/verb.exc  
  inflating: /usr/share/nltk_data/co

In [4]:
data = pd.read_csv('/kaggle/input/human-vs-llm-text-corpus/data.csv')

In [5]:
data.describe()

,prompt_id,text_length,word_count
count,788922.000000,788922.000000,788922.000000
mean,420.069514,3123.374844,497.983404
std,1381.285340,4963.074433,720.866585
min,0.000000,105.000000,25.000000
25%,0.000000,647.000000,110.000000
50%,0.000000,1921.000000,324.000000
75%,0.000000,3910.000000,624.000000
max,9913.000000,890119.000000,71543.000000


In [6]:
# Примерный план решения задачи генерации следующего слова на основе нескольких предыдущих слов
# Количество предыдущих слов - 5
# На основе этой модели будем генерировать предложения в цикле while true 
# с условием на длину предложения не больше max_length
# Модель будем просто представлять собой embedding -> lstm layer -> linear layer -> softmax
# Обучим наши собственные embeddings и измерим на них nll_avg_loss
# Обучим модель на предобученных embeddings и измерим nll_avg_loss

In [7]:
texts = data.loc[:].text.tolist()

In [8]:
def tokenize_lemmatize(texts, tokenizer, lemmatizer):
    tokenized = [tokenizer.tokenize(sentence) for sentence in texts]
    lemmatized = [[lemmatizer.lemmatize(word) for word in sent] for sent in tokenized]
    return lemmatized
tokenizer = WordPunctTokenizer()
lemmatizer = WordNetLemmatizer()
preprocessed = tokenize_lemmatize(texts[:100], tokenizer, lemmatizer)


In [9]:
vocab = set([word for sent in preprocessed for word in sent])
vocab.add('<end>')

In [10]:
itos = {i : word for i, word in enumerate(vocab)}
stoi = {word : i for i, word in itos.items()}

In [229]:
# creating X and y tensors
def build_dataset(preprocessed):
    X, y = [], []
    window_size = 5
    for sent in preprocessed:
        sent.append('<end>')
        left = 0
        for right in range(window_size, len(sent)):
            context = [stoi[word] for word in sent[left : right]]
            positive_target = stoi[sent[right]]
            X.append(context)
            y.append([positive_target, 1])
            left += 1
    X, y = torch.tensor(X), torch.tensor(y)
    # negative sampling
    random_indices = torch.randperm(X.shape[0])[:len(preprocessed) * 20]
    negative_samples = X[random_indices]
    negative_targets = torch.zeros(size = (negative_samples.shape[0], 2))
    negative_targets[:, 0] = torch.randint(0, len(vocab) - 1, size = (negative_samples.shape[0],))
    
    return torch.concat((X, negative_samples), dim = 0), torch.concat((y, negative_targets), dim = 0).int()
X, y = build_dataset(preprocessed)
    

In [230]:
def nll(inputs, targets):
    # inputs is the result after linear layer
    # inputs: batch_size x C, C - number of targets
    # targets: value where 0<=value<C, index of target
    counts = inputs.exp()
    preds = counts / counts.sum(1, keepdims = True)
    return -(preds[torch.arange(preds.shape[0]), targets]).log().mean().item()
def nll_logprobs(log_probs, targets):
    return -log_probs[torch.arange(log_probs.shape[0]), targets].mean().item()

def cross_entropy_loss(preds, targets):
    # targets will be in format [target_idx, target_class]: target_class {0, 1}
    l1, l2 = targets[targets[:, 1] == 1].shape[0], targets[targets[:, 1] == 0].shape[0]
    # Calculate losses for ones and zeros
    # Separate targets based on target_class
    ones_targets = targets[targets[:, 1] == 1][:, 0]
    zeros_targets = targets[targets[:, 1] == 0][:, 0]
    ones_loss = zeros_loss = 0
    e = 0.00001
    if len(ones_targets) > 0:
        ones_loss = -(preds[torch.arange(len(ones_targets)), ones_targets] + e).log().mean()

    if len(zeros_targets) > 0:
        zeros_loss = (preds[torch.arange(len(zeros_targets)), zeros_targets] + e).log().mean()
    return (ones_loss + zeros_loss) / 2

In [231]:
# create embedding function
def embedding(x, C):
    return C[x]


In [238]:
C = torch.randn(len(vocab), 50) 
input_dim = 50
hidden_dim = 100
seq_length = 5
batch_size = 64
H = torch.randn(seq_length, batch_size, hidden_dim); h_0 = torch.randn(batch_size, hidden_dim)
c_0 = torch.randn(1, batch_size, hidden_dim)
W_x = torch.randn(4 * hidden_dim, input_dim).T
W_h = torch.randn(4 * hidden_dim, hidden_dim).T
b = torch.randn(4 * hidden_dim)
W, b2 = torch.randn(100, len(vocab)), torch.randn(len(vocab))

In [239]:
# main part of model, LSTM layer
def lstm_cell(x, h, c, W_h, W_x, b):
    # we are also going to handle batches in this implementation of lstm
    i, f, g, o = torch.split(x @ W_x + h @ W_h + b, split_size_or_sections = h.shape[1] , dim = 1)
    #print((x @ W_x + h @ W_h + b).shape)
    i, f, g, o = F.sigmoid(i), F.sigmoid(f), torch.tanh(g), F.sigmoid(o)
    c = f * c + i * g
    h = o * torch.tanh(c).squeeze(0)
    return h, c
def lstm(X, h = h_0, c = c_0, W_h = W_h, W_x = W_x, b = b):
    for i in range(X.shape[0]):
        h, c = lstm_cell(X[i], h, c, W_h, W_x, b)
        H[i] = h
    return H, c


In [240]:
parameters = [C, W, b2, b, W_x, W_h]
for p in parameters:
    p.requires_grad = True
for p in parameters:
    p.grad = None

In [244]:
losses = []
for i in range(100):
    indices = torch.randint(0, X.shape[0], (64, ))
    batch = embedding(X[indices], C).permute(1, 0, 2)
    hidden = lstm(batch)[0][-1]
    logits = hidden @ W + b2
    logits = logits - logits.max(dim = 1)[0].view(-1, 1) # optimizing to reduce infinities
    counts = logits.exp()
    pred = counts / counts.sum(1, keepdims = True)
    loss = cross_entropy_loss(pred, y[indices])
    losses.append(loss.item())
    for p in parameters:
        p.grad = None
    loss.backward()
    for p in parameters:
        p.data -= 0.1 * p.grad
# RuntimeError: Trying to backward through the graph a second time (or directly access saved tensors after they have already been freed). 
# Saved intermediate values of the graph are freed when you call .backward() or autograd.grad(). 
# Specify retain_graph=True if you need to backward through the graph a second time or if you need to access saved tensors after calling backward.
# add Codeadd Markdown

RuntimeError: Trying to backward through the graph a second time (or directly access saved tensors after they have already been freed). Saved intermediate values of the graph are freed when you call .backward() or autograd.grad(). Specify retain_graph=True if you need to backward through the graph a second time or if you need to access saved tensors after calling backward.

In [216]:
r = range(len(losses))
plt.plot(r, losses)